# Globale trends
## Data importeren

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date,datetime,timedelta
from matplotlib.ticker import ScalarFormatter
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_rgba_array

In [ ]:
gemeentes = pd.read_csv('https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_cumulatief.csv', parse_dates=['Date_of_report'], sep=';').drop(columns=['Hospital_admission', 'Deceased'])
gemeentes.loc[gemeentes.Municipality_code.isnull(), 'Municipality_code'] = 'Onbekend'
gemeentes.loc[gemeentes.Province.isnull(), 'Province'] = 'Onbekend'
gemeentes.loc[gemeentes.Province == 'Friesland', 'Province'] = 'Fryslân'
gemeentes = gemeentes.fillna(0)
gemeentes

In [ ]:
totalPositiveTestRatio = pd.read_csv('https://raw.githubusercontent.com/mzelst/covid-19/master/data-dashboards/percentage-positive-daily-national.csv', parse_dates=['date'])
totalPositiveTestRatio['infected_percentage_7d'] =  totalPositiveTestRatio['values.infected_percentage'].rolling(7).mean()
totalPositiveTestRatio['total_7d'] =  totalPositiveTestRatio['values.tested_total'].rolling(7).mean()
totalPositiveTestRatio['positive_7d'] =  totalPositiveTestRatio['values.infected'].rolling(7).mean()
totalPositiveTestRatio

## Data voorbereiden

In [ ]:
from datetime import datetime
totals = gemeentes.groupby(by=['Date_of_report']).sum().diff().reset_index()
totals['rolling'] = totals.Total_reported.rolling(window=7).mean()
#totals['rolling_100k'] = totals['rolling'] * (100000 / 17454856) 
totals['rolling14'] = totals.Total_reported.rolling(window=14).mean()
#totals['rolling14_100k'] = totals.rolling14 *  (100000 / 17454856)
totals

In [ ]:
perWeek = totals.copy()
perWeek['week'] = perWeek.Date_of_report.dt.isocalendar().week
perWeek['year'] = perWeek.Date_of_report.dt.isocalendar().year
weekTotals = perWeek.groupby(by=['year', 'week']).mean()
weekTotals.drop(columns=['rolling', 'rolling14'], inplace=True)
weekTotals.rename(columns={'Total_reported': 'WeekTotalReported', 'Hospital_admission': 'WeekHospitalAdmission', 'Deceased' : 'WeekDeceased'}, inplace=True)
perWeek = perWeek.set_index(['year','week']).join(weekTotals).reset_index()
perWeek['Reported_ratio'] = (perWeek.Total_reported / perWeek.WeekTotalReported)

dayOfWeek={0:'Maandag', 1:'Dinsdag', 2:'Woensdag', 3:'Donderdag', 4:'Vrijdag', 5:'Zaterdag', 6:'Zondag'}
perWeek['weekday'] = perWeek.Date_of_report.dt.dayofweek.map(dayOfWeek)
perWeek['month'] = perWeek.Date_of_report.dt.month
perWeek['date'] = pd.to_datetime(perWeek.year.apply(str) + '-W' + perWeek.week.apply(str) + '-7', format='%G-W%V-%u')
display(perWeek)

dayToWeekAvg = totals.copy()
dayToWeekAvg['week'] = dayToWeekAvg.Date_of_report.dt.isocalendar().week
dayToWeekAvg['year'] = dayToWeekAvg.Date_of_report.dt.isocalendar().year
dayToWeekAvg['date'] = pd.to_datetime(dayToWeekAvg.year.apply(str) + '-W' + dayToWeekAvg.week.apply(str) + '-7', format='%G-W%V-%u')
dayToWeekAvg['RatioWeekAvg'] = dayToWeekAvg.Total_reported / dayToWeekAvg['rolling'].shift(1)
dayToWeekAvg['AboveAverage'] = dayToWeekAvg.RatioWeekAvg > 1
dayToWeekAvg['weekday'] = dayToWeekAvg.Date_of_report.dt.dayofweek.map(dayOfWeek)
display(dayToWeekAvg)

## Plots configureren

In [ ]:
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1, rc={"lines.linewidth": 3, "font.size":10,"axes.titlesize":24,"axes.labelsize":18})
sns.set_palette(sns.color_palette(['#CC6677', '#332288', '#DDCC77', '#117733', '#88CCEE', \
                    '#882255', '#44AA99', '#999933', '#AA4499', '#DDDDDD', \
                    '#000000']))

def discretemap(colormap, hexclrs):
    """
    Produce a colormap from a list of discrete colors without interpolation.
    """
    clrs = to_rgba_array(hexclrs)
    clrs = np.vstack([clrs[0], clrs, clrs[-1]])
    cdict = {}
    for ki, key in enumerate(('red','green','blue')):
        cdict[key] = [ (i/(len(clrs)-2.), clrs[i, ki], clrs[i+1, ki]) for i in range(len(clrs)-1) ]
    return LinearSegmentedColormap(colormap, cdict)

def __rainbow_WhBr():
    """
    Define colormap 'rainbow_WhBr'.
    """
    clrs = ['#E8ECFB', '#DDD8EF', '#D1C1E1', '#C3A8D1', '#B58FC2',
            '#A778B4', '#9B62A7', '#8C4E99', '#6F4C9B', '#6059A9',
            '#5568B8', '#4E79C5', '#4D8AC6', '#4E96BC', '#549EB3',
            '#59A5A9', '#60AB9E', '#69B190', '#77B77D', '#8CBC68',
            '#A6BE54', '#BEBC48', '#D1B541', '#DDAA3C', '#E49C39',
            '#E78C35', '#E67932', '#E4632D', '#DF4828', '#DA2222',
            '#B8221E', '#95211B', '#721E17', '#521A13']
    return LinearSegmentedColormap.from_list("Rainbo_WhBr", clrs)
    #self.cmap.set_bad('#666666')

def __iridescent():
    """
    Define colormap 'iridescent'.
    """
    clrs = ['#FEFBE9', '#FCF7D5', '#F5F3C1', '#EAF0B5', '#DDECBF',
            '#D0E7CA', '#C2E3D2', '#B5DDD8', '#A8D8DC', '#9BD2E1',
            '#8DCBE4', '#81C4E7', '#7BBCE7', '#7EB2E4', '#88A5DD',
            '#9398D2', '#9B8AC4', '#9D7DB2', '#9A709E', '#906388',
            '#805770', '#684957', '#46353A']
    return LinearSegmentedColormap.from_list("iridescent", clrs)
    #self.cmap.set_bad('#999999')

def __YlOrBr():
    """
    Define colormap 'YlOrBr'.
    """
    clrs = ['#FFFFE5', '#FFF7BC', '#FEE391', '#FEC44F', '#FB9A29',
            '#EC7014', '#CC4C02', '#993404', '#662506']
    return LinearSegmentedColormap.from_list('YlOrBr', clrs)
    #self.cmap.set_bad('#888888')

def __BuRd():
    """
    Define colormap 'BuRd'.
    """
    clrs = ['#2166AC', '#4393C3', '#92C5DE', '#D1E5F0', '#F7F7F7',
            '#FDDBC7', '#F4A582', '#D6604D', '#B2182B']
    return LinearSegmentedColormap.from_list('BuRd', clrs)

def __rainbow_discrete(self, lut=None):
    """
    Define colormap 'rainbow_discrete'.
    """
    clrs = ['#E8ECFB', '#D9CCE3', '#D1BBD7', '#CAACCB', '#BA8DB4',
            '#AE76A3', '#AA6F9E', '#994F88', '#882E72', '#1965B0',
            '#437DBF', '#5289C7', '#6195CF', '#7BAFDE', '#4EB265',
            '#90C987', '#CAE0AB', '#F7F056', '#F7CB45', '#F6C141',
            '#F4A736', '#F1932D', '#EE8026', '#E8601C', '#E65518',
            '#DC050C', '#A5170E', '#72190E', '#42150A']
    indexes = [[9], [9, 25], [9, 17, 25], [9, 14, 17, 25], [9, 13, 14, 17,
            25], [9, 13, 14, 16, 17, 25], [8, 9, 13, 14, 16, 17, 25], [8,
            9, 13, 14, 16, 17, 22, 25], [8, 9, 13, 14, 16, 17, 22, 25, 27],
            [8, 9, 13, 14, 16, 17, 20, 23, 25, 27], [8, 9, 11, 13, 14, 16,
            17, 20, 23, 25, 27], [2, 5, 8, 9, 11, 13, 14, 16, 17, 20, 23,
            25], [2, 5, 8, 9, 11, 13, 14, 15, 16, 17, 20, 23, 25], [2, 5,
            8, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25], [2, 5, 8, 9, 11,
            13, 14, 15, 16, 17, 19, 21, 23, 25, 27], [2, 4, 6, 8, 9, 11,
            13, 14, 15, 16, 17, 19, 21, 23, 25, 27], [2, 4, 6, 7, 8, 9, 11,
            13, 14, 15, 16, 17, 19, 21, 23, 25, 27], [2, 4, 6, 7, 8, 9, 11,
            13, 14, 15, 16, 17, 19, 21, 23, 25, 26, 27], [1, 3, 4, 6, 7, 8,
            9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 26, 27], [1, 3, 4,
            6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 21, 23, 25, 26,
            27], [1, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20,
            22, 24, 25, 26, 27], [1, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15,
            16, 17, 18, 20, 22, 24, 25, 26, 27, 28], [0, 1, 3, 4, 6, 7, 8,
            9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 22, 24, 25, 26, 27, 28]]
    if lut == None or lut < 1 or lut > 23:
        lut = 22
    self.cmap = discretemap(self.cname, [ clrs[i] for i in indexes[lut-1] ])
    if lut == 23:
        self.cmap.set_bad('#777777')
    else:
        self.cmap.set_bad('#FFFFFF')

In [ ]:
def tuftefy(ax):
    """Remove spines and tick position markers to reduce ink."""
    # 
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(True)
    ax.spines["bottom"].set_color('grey')

    ax.grid(color="w", alpha=0.5)
    ax.get_yaxis().grid(True)
    ax.get_xaxis().grid(False)

def fancy_titles(t1, t2, ax=None):
  if ax:
    ax.set_title(t1, loc='left', fontsize=18)
    ax.set_title(t2, loc='right', fontsize=13, color='grey')
  else:
    plt.title(t1, loc='left', fontsize=18)
    plt.title(t2, loc='right', fontsize=13, color='grey')

## Plots
### Week gemiddeld

In [ ]:
#sns.set_context("talk")
fig = plt.gcf()
fig.set_size_inches(15, 6)
e =sns.violinplot(x="weekday", linewidth=2, y="Reported_ratio", data=perWeek[perWeek.Date_of_report >= '2020-09-01'], order=["Maandag", "Dinsdag", "Woensdag", "Donderdag", "Vrijdag", "Zaterdag", "Zondag"], inner=None, color='#ccc', scale_hue=False, cut=0.5, bw=0.4)
e =sns.stripplot(x="weekday", y="Reported_ratio", data=perWeek[perWeek.Date_of_report >= '2020-09-01'],order=["Maandag", "Dinsdag", "Woensdag", "Donderdag", "Vrijdag", "Zaterdag", "Zondag"], color='black', alpha=0.5)
e.set(ylabel='Verhouding tov weekgemiddelde', title='Verhouding tussen positieve COVID-19 testen en de weekdag', xlabel='')
plt.title("bron: RIVM update " + (totals.Date_of_report.to_list()[-1]).strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=0.01)
#tuftefy(e)
e.yaxis.grid(True)
sns.despine(bottom=True)


In [ ]:
fig = plt.gcf()
fig.set_size_inches(15, 6)
e = sns.countplot(x="weekday", hue="AboveAverage", data=dayToWeekAvg[dayToWeekAvg.Date_of_report >= '2020-09-01'], order=["Maandag", "Dinsdag", "Woensdag", "Donderdag", "Vrijdag", "Zaterdag", "Zondag"])

e.set(ylabel='Aantal weken', title='Aantal keer dag boven week gemiddelde', xlabel='Weekdag')
plt.title("bron: RIVM update " + (totals.Date_of_report.to_list()[-1]).strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=0.0)
e.yaxis.grid(True)
sns.despine(bottom=True)

In [ ]:
fig = plt.gcf()
fig.set_size_inches(15, 6)
ax = sns.lineplot(x='date', y='Reported_ratio', hue='weekday', style='weekday', linewidth=2, hue_order=['Maandag', 'Dinsdag', 'Woensdag', 'Donderdag', 'Vrijdag', 'Zaterdag', 'Zondag'], data=perWeek[perWeek.Date_of_report >= '2021-01-01'])
ax.set_ylim(0.5,1.5)
plt.axhline(y=0.8, ls=':', c='.8', linewidth=1, zorder=1)
plt.axhline(y=1.0, ls='-', c='.85', linewidth=1, zorder=1)
plt.axhline(y=1.2, ls=':', c='.8', linewidth=1, zorder=1)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%V'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.SUNDAY, interval=3))
ax.set(ylabel='Verhouding tov weekgemiddelde', title='Verhouding tussen positieve COVID-19 testen en de weekdag', xlabel='Week')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
tuftefy(ax)

In [ ]:
fig = plt.gcf()
fig.set_size_inches(15, 6)
ax = sns.lineplot(x='date', y='RatioWeekAvg', hue='weekday', style='weekday', linewidth=2, hue_order=['Maandag', 'Dinsdag', 'Woensdag', 'Donderdag', 'Vrijdag', 'Zaterdag', 'Zondag'], data=dayToWeekAvg[dayToWeekAvg.Date_of_report >= '2021-01-01'])
ax.set_ylim(0.5,1.5)
plt.axhline(y=0.8, ls=':', c='.8', linewidth=1, zorder=1)
plt.axhline(y=1.0, ls='-', c='.85', linewidth=1, zorder=1)
plt.axhline(y=1.2, ls=':', c='.8', linewidth=1, zorder=1)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%V'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.SUNDAY, interval=3))
ax.set(ylabel='Verhouding tov weekgemiddelde', title='Verhouding tussen positieve COVID-19 testen en de 7d gemiddelde', xlabel='Week')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
tuftefy(ax)

In [ ]:

fig = plt.figure()
fig.set_size_inches(15, 10)
#display(dayToWeekAvg[dayToWeekAvg.weekday == "Dinsdag"])

ax = sns.lineplot(data=totals, x='Date_of_report', y='rolling', color='#004488', label='Aantal positief')
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax2 = ax.twinx()
sns.lineplot(data=dayToWeekAvg[dayToWeekAvg.weekday == "Dinsdag"], x='Date_of_report', y='RatioWeekAvg', color='#ddaa33', label='Ratio dinsdag tov weekgemiddelde', ax = ax2)
plt.legend(bbox_to_anchor=(0.9, 1),borderaxespad=0)
#plt.axhline(y=0.8, ls=':', c='.8', linewidth=1, zorder=1)
plt.axhline(y=1.0, ls='-', c='.85', linewidth=1, zorder=1)
#plt.axhline(y=1.2, ls=':', c='.8', linewidth=1, zorder=1)
#sns.lineplot(data=totals[totals.Date_of_report > '2020-06-01'], x='Date_of_report',y='rolling', color='red',  ax=ax2)
ax.set_ylabel("Aantal testen")
ax2.set_ylabel("Positieve testen")
ax.set_xlabel("")
ax.set_title('Voortschrijdend gemiddelde (7d) relatie tot de dinsdag van de week', pad=30)
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%V'))
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
#ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.SUNDAY, interval=2))
ax.set(ylim=(0, None))
ax2.set(ylim=(0, None))
#plt.axvline(date(2020,10,14), ls=":", c=".5")
#plt.axvline(date(2020,12,18), ls=":", c=".5")
#plt.axvline(date(2021,2,8), ls=":", c=".5")
#plt.axhline(0.0, ls="-", c=".8", zorder=1)
#g = g.map(plt.axvline, x=date(2020,10,14), ls=":", c=".5")
plt.title("bron: RIVM data t/m " + (totals.Date_of_report.to_list()[-1]).strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=0.01)


tuftefy(ax2)
tuftefy(ax)

### Totals

In [ ]:
fig = plt.figure()
fig.set_size_inches(15, 10)

#ax = sns.lineplot(data=totalPositiveTestRatio[totalPositiveTestRatio.date >= '2020-08-01'], x='date', y='total_7d', color='#004488', label='Aantal testen')

ax = sns.lineplot(data=totalPositiveTestRatio, x='date', y='total_7d', color='#004488', label='Aantal testen')
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax2 = ax.twinx()
#sns.lineplot(data=totalPositiveTestRatio[totalPositiveTestRatio.date >= '2020-08-01'], x='date', y='positive_7d', color='#ddaa33', label='Aantal positief', ax = ax2)
sns.lineplot(data=totalPositiveTestRatio, x='date', y='positive_7d', color='#ddaa33', label='Aantal positief', ax = ax2)
plt.legend(bbox_to_anchor=(0.9, 1),borderaxespad=0)
#sns.lineplot(data=totals[totals.Date_of_report > '2020-06-01'], x='Date_of_report',y='rolling', color='red',  ax=ax2)
ax.set_ylabel("Aantal testen")
ax2.set_ylabel("Positieve testen")
ax.set_xlabel("")
ax.set_title('Voortschrijdend gemiddelde (7d) van aantal uitgevoerde en positieve testen', pad=30)
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%V'))
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
#ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.SUNDAY, interval=2))
ax.set(ylim=(0, None))
ax2.set(ylim=(0, None))
#plt.axvline(date(2020,10,14), ls=":", c=".5")
#plt.axvline(date(2020,12,18), ls=":", c=".5")
#plt.axvline(date(2021,2,8), ls=":", c=".5")
#plt.axhline(0.0, ls="-", c=".8", zorder=1)
#g = g.map(plt.axvline, x=date(2020,10,14), ls=":", c=".5")
plt.title("bron: Dashboard data t/m " + (totalPositiveTestRatio.date.to_list()[-1]).strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=0.0)


tuftefy(ax)
tuftefy(ax2)